# Estimación del estadio puberal de Tanner para añadir al M5 y afinarlo 

In [15]:
import pandas as pd
import pymc as pm
import arviz as az
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import pytensor  
from patsy import dmatrix


In [16]:
# read data
df = pd.read_csv("../data/clean/clean2_final_nutricion_salud.csv")

---

In [17]:
# Filtramos observaciones con datos completos de talla, edad y sexo
df_model = df[["talla_cm", "edad_anios_calc", "sexo", "peso_kg", "municipio", "id_persona"]].dropna()

# Establecemos 'id_persona' como índice del DataFrame
df_model = df_model.set_index("id_persona")

# Codificamos correctamente sexo 
# Recodeamos sexo: 0 = mujer, 1 = hombre
df_model["sexo"] = df_model["sexo"].map({2: 0, 1: 1})

# Confirmamos los tipos de variables y que no haya NAs
df_model.info()
df_model.describe()
df_model["sexo"].value_counts()
df_model["municipio"].nunique()

<class 'pandas.core.frame.DataFrame'>
Index: 25355 entries, 100001_3 to 70336_4
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   talla_cm         25355 non-null  float64
 1   edad_anios_calc  25355 non-null  float64
 2   sexo             25355 non-null  int64  
 3   peso_kg          25355 non-null  float64
 4   municipio        25355 non-null  float64
dtypes: float64(4), int64(1)
memory usage: 1.2+ MB


177

In [18]:
# Codificamos cada municipio con un índice entero único (de 0 a n_grupos - 1)
df_model["municipio_idx"] = pd.Categorical(df_model["municipio"]).codes

# Verificamos que se creó correctamente
df_model[["municipio", "municipio_idx"]].drop_duplicates().sort_values("municipio_idx").head()


,municipio,municipio_idx
id_persona,,
100001_3,1.0,0
11276_3,2.0,1
110044_5,3.0,2
100075_5,4.0,3
100121_6,5.0,4


---

In [19]:
# Centrar la edad y guardar la media: 
# Esto ayuda a que el intercepto sea interpretable (talla promedio en la edad media) y mejora la estabilidad numérica del muestreo.
edad_mean = df_model["edad_anios_calc"].mean()
df_model["edad_c"] = df_model["edad_anios_calc"] - edad_mean

print(f"Edad media en la muestra: {edad_mean:.3f} años")
df_model[["edad_anios_calc", "edad_c"]].head()


Edad media en la muestra: 6.266 años


,edad_anios_calc,edad_c
id_persona,,
100001_3,8.695414,2.429254
100006_6,11.211499,4.945339
100008_10,7.561944,1.295784
100009_3,6.277892,0.011732
100010_7,8.145106,1.878946


---

## Splines

In [20]:
# Construcción de B-splines cúbicos para la edad
# (A) Hiperparámetro de flexibilidad
df_spline = 5        # Empezamos con 5; luego probamos 6 o 7 si hace falta.

# (B) Matriz de B-splines cúbicos de la edad (sin intercepto)
X_spline = dmatrix(
    "bs(edad, df=df_spline, degree=3, include_intercept=False) - 1",
    {"edad": df_model["edad_anios_calc"].values},
    return_type="dataframe"
)

# (C) A NumPy para PyMC
X_s = X_spline.to_numpy()

# (D) Chequeo rápido
n_obs, n_s = X_s.shape
print("X_s shape:", X_s.shape)   # (n_obs, n_bases)


X_s shape: (25355, 5)


In [21]:
# Variable objetivo y predictores “clásicos”
y        = df_model["talla_cm"].to_numpy()
sexo     = df_model["sexo"].to_numpy().astype(int)  # 0 = mujer, 1 = hombre

# Edad centrada (importante para estabilidad y para interpretar el intercepto)
edad_c   = (df_model["edad_anios_calc"] - df_model["edad_anios_calc"].mean()).to_numpy()

# Índice entero de municipio por observación (0..n_muni-1)
muni_idx = df_model["municipio_idx"].to_numpy().astype(int)

# Tamaños
n_obs_chk = y.shape[0]
n_muni    = int(muni_idx.max()) + 1

# Chequeos
assert X_s.shape[0] == n_obs_chk, "X_s debe tener tantas filas como observaciones."
print(f"n_obs={n_obs_chk} | n_s(bases)={n_s} | n_muni={n_muni}")


n_obs=25355 | n_s(bases)=5 | n_muni=177


---

## Estimación estadio puberal Tanner con modelos latentes probabilísticos (LVPM)

Qué son los estadios Tanner?

Se usan clínicamente para evaluar el desarrollo físico en la pubertad, con 5 fases (Tanner 1 a 5), basadas en características sexuales secundarias: desarrollo mamario/genital y vello púbico. Son el “estándar de oro” en pediatría

### Modelos latentes probabilísticos aplicados a Tanner

Un modelo latente parte de la idea de que, además de las variables que observamos (edad, sexo, talla, IMC), existe una **variable oculta** que no se mide directamente pero que explica parte del patrón en los datos.  
En el contexto de crecimiento infantil, esa variable oculta corresponde al **estadio puberal de Tanner (I–V)**.

#### 1. Probabilidades de estadio
El modelo no asigna a cada niño un estadio fijo, sino que estima **probabilidades** para cada uno de los cinco estadios.  
Estas probabilidades dependen de características observadas como la edad, el sexo y el IMC.  
Ejemplo: un niño de 13 años podría tener:  
- 10% Tanner II  
- 40% Tanner III  
- 50% Tanner IV  

#### 2. Distribuciones por estadio
Cada estadio Tanner tiene su propia **distribución de talla esperada**.  
- Tanner I: estaturas más bajas.  
- Tanner V: estaturas más altas.  
- Los estadios intermedios representan el crecimiento acelerado del estirón puberal.  

De esta forma, si un niño está en Tanner III, se espera que su talla se distribuya alrededor de un valor típico para ese estadio, con cierta variabilidad natural.

#### 3. Mezcla probabilística
Como no sabemos en qué estadio se encuentra exactamente cada niño, el modelo combina todas las distribuciones ponderadas por las probabilidades:  

**Prob(talla) = Prob(Tanner I)·Distribución I + Prob(Tanner II)·Distribución II + ... + Prob(Tanner V)·Distribución V**

Esto refleja que la talla observada no proviene de una única categoría fija, sino de una mezcla que captura la **incertidumbre** del estado puberal.

#### 4. Interpretación
- Las **probabilidades** responden a: “¿Qué tan probable es que este niño esté en cada estadio Tanner?”.  
- Las **distribuciones por estadio** responden a: “Si el niño estuviera en este estadio, ¿cuál sería la talla típica?”.  
- La **mezcla final** responde a: “Dada su edad y características, ¿cómo se distribuye la talla observada?”.  

#### Ventajas
- No fuerza a clasificar en un único estadio.  
- Captura la incertidumbre en edades de transición.  
- Representa mejor la heterogeneidad real del crecimiento que un modelo lineal simple.  
- Permite aproximar el Tanner aunque no haya mediciones clínicas directas.


---

#### Añadir una variable latente puberal al M5 primero con 2 estados: PRE y POST.

In [22]:
# --- Preparación mínima (solo si no está corrido arriba) ---

y        = df_model["talla_cm"].to_numpy()
sexo     = df_model["sexo"].map({2:0, 1:1}).to_numpy().astype(int)  # 0=mujer, 1=hombre
edad     = df_model["edad_anios_calc"].to_numpy()
edad_c   = edad - edad.mean()
edad_cs  = edad_c / (edad_c.std() + 1e-8)

muni_idx = pd.Categorical(df_model["municipio"]).codes.astype(int)
n_muni   = int(muni_idx.max()) + 1

# X_s: tu matriz de splines tiene que ser array float
X_s = X_s.astype(float)

# Chequeos de tamaño
n_obs = y.shape[0]
n_s   = X_s.shape[1]
assert X_s.shape[0] == n_obs, "X_s debe tener tantas filas como observaciones."
assert sexo.shape[0] == n_obs and edad_cs.shape[0] == n_obs and muni_idx.shape[0] == n_obs, "Vectores mal alineados."

print(f"n_obs={n_obs} | n_s={n_s} | n_muni={n_muni}")


n_obs=25355 | n_s=5 | n_muni=177


/var/folders/8v/p9jmdytd6y36kdsq55l4_cqw0000gn/T/ipykernel_42476/758676489.py:4: RuntimeWarning: invalid value encountered in cast
  sexo     = df_model["sexo"].map({2:0, 1:1}).to_numpy().astype(int)  # 0=mujer, 1=hombre


Para cada observación, el modelo calcula una probabilidad de POST: p_post (entre 0 y 1).

p_post se modela con una logística que aumenta con la edad (y puede depender de sexo).

Parámetros:

g0: intercepto (prob POST en la edad media).

g_ed: cuánto sube la prob POST cuando la edad sube 1 desviación estándar.

g_sx: cambio de prob POST por ser hombre (opcional).

La talla no “elige” PRE o POST; en cambio, se modela como mezcla:

Componente PRE: media = mu_base + delta_pre

Componente POST: media = mu_base + delta_post

Para que los dos “ajustes” sean identificables, les imponemos suma cero:
delta_pre + delta_post = 0.
Así, si delta_post > 0 ⇒ POST está por encima de la base; PRE, por debajo.

# Muestreo estratificado

Tarda 12+ horas en correr, vamos a trabajar con una muestra hasta tener el modelo. 
Necesitamos hacer muestreo estratificado para no tener datos desbalanceados --> muestrear por estratos y proporcional al tamaño de los grupos

In [23]:
# Asegura sexo binario 0/1
if set(df_model["sexo"].unique()) - {0,1}:
    df_model = df_model.copy()
    df_model["sexo"] = df_model["sexo"].map({2:0, 1:1})

# Mantén solo columnas clave y sin NA
cols_req = ["talla_cm", "edad_anios_calc", "sexo", "municipio"]
df_base = df_model[cols_req].dropna().copy()

print("Filas disponibles:", len(df_base))
assert set(df_base["sexo"].unique()) <= {0,1}, "Columna 'sexo' debe ser 0/1."


Filas disponibles: 25355


In [24]:
# 1) Bins sin vacíos (hasta 13 años)
age_bins   = [-np.inf, 5, 7, 9, 11, 13]
age_labels = [f"({age_bins[i]},{age_bins[i+1]}]" for i in range(len(age_bins)-1)]

df_b = df_model[["talla_cm","edad_anios_calc","sexo","municipio"]].dropna().copy()
if set(df_b["sexo"].unique()) - {0,1}:
    df_b["sexo"] = df_b["sexo"].map({2:0, 1:1})

df_b["edad_bin"] = pd.cut(df_b["edad_anios_calc"], bins=age_bins, labels=age_labels, include_lowest=True)

# 2) Distribución original 
dist_orig = (df_b
    .groupby(["sexo","edad_bin"], observed=True)
    .size().rename("n").reset_index())

print("Distribución original por estrato (sexo x edad_bin):")
print(dist_orig.pivot(index="edad_bin", columns="sexo", values="n").fillna(0))


Distribución original por estrato (sexo x edad_bin):
sexo         0     1
edad_bin            
(-inf,5]  4676  4788
(5,7]     2310  2268
(7,9]     2455  2468
(9,11]    2183  2273
(11,13]    961   973


In [25]:
# === A. Muestreo estratificado exacto con scikit-learn === explicación de CHAT GPT
# Idea: queremos una SUBMUESTRA de tamaño N_SUB que conserve, lo mejor posible,
#       las proporciones originales de cada ESTRATO. Un "estrato" es una
#       combinación de variables (aquí: SEXO × RANGO DE EDAD).

from sklearn.model_selection import StratifiedShuffleSplit

N_SUB = 6000  # Tamaño deseado de la submuestra (ajustable)

# 1) Definimos la "etiqueta de estrato" para cada fila.
#    • Unimos sexo y el bin de edad en un string, por ejemplo: "0_(9,11]" o "1_(7,9]"
#    • Esto le dice al algoritmo en qué estrato está cada observación.
estrato = df_b["sexo"].astype(str) + "_" + df_b["edad_bin"].astype(str)

# 2) Creamos el "divisor estratificado".
#    • n_splits=1: queremos una sola partición (no un k-fold).
#    • test_size=N_SUB: el TAMAÑO del subconjunto que sacaremos (aquí "test" es la submuestra).
#    • random_state=42: semilla para que el resultado sea reproducible.
sss = StratifiedShuffleSplit(n_splits=1, test_size=N_SUB, random_state=42)

# 3) Ejecutamos el split.
#    • sss.split(X, y) recibe:
#         - X: la tabla completa (no se usa su contenido, solo el tamaño).
#         - y: las etiquetas de estrato (aquí 'estrato').
#    • Devuelve dos arreglos de índices: (idx_train, idx_test).
#      Usaremos idx_test como nuestra SUBMUESTRA, porque le dijimos que "test_size = N_SUB".
_, idx_sub = next(sss.split(df_b, estrato))

# 4) Extraemos la submuestra usando los índices calculados.
#    • .iloc[idx_sub] selecciona esas filas.
#    • .copy() para tener un DataFrame independiente (evita advertencias de pandas).
df_sub = df_b.iloc[idx_sub].copy()

# 5) Revisamos tamaño y distribución de la submuestra.
#    • Debe salir (N_SUB, n_columnas).
#    • La distribución por (sexo × edad_bin) debería ser MUY parecida a la original.
print("Submuestra final:", df_sub.shape)

# observed=True: usa solo categorías que aparecen (evita warning de pandas).
print(df_sub.groupby(["sexo","edad_bin"], observed=True).size())


Submuestra final: (6000, 5)
sexo  edad_bin
0     (-inf,5]    1106
      (5,7]        547
      (7,9]        581
      (9,11]       517
      (11,13]      227
1     (-inf,5]    1133
      (5,7]        537
      (7,9]        584
      (9,11]       538
      (11,13]      230
dtype: int64


In [26]:
# ====== A) Reindexar municipios y preparar arrays ======

# 1) Reindexar municipios (solo los que aparecen en la SUBMUESTRA)
df_sub = df_sub.copy()
df_sub["municipio_idx_sub"] = pd.Categorical(df_sub["municipio"]).codes

# 2) Arreglos NumPy que usará PyMC
y_sub     = df_sub["talla_cm"].to_numpy().astype(float)          # variable objetivo (cm)
edad_sub  = df_sub["edad_anios_calc"].to_numpy().astype(float)   # edad (años)
sexo_sub  = df_sub["sexo"].to_numpy().astype(int)                # 0 = mujer, 1 = hombre
muni_sub  = df_sub["municipio_idx_sub"].to_numpy().astype(int)   # índice municipal 0..J-1

# 3) Versiones centrada y estandarizada de la edad
#    - edad_c_sub: centrada => intercepto ~ talla a la edad media
#    - edad_cs_sub: centrada + estandarizada => estable numéricamente para la compuerta (logística)
edad_c_sub  = edad_sub - edad_sub.mean()
edad_cs_sub = (edad_c_sub - edad_c_sub.mean()) / edad_c_sub.std()

# 4) Chequeos rápidos (defensivos)
assert set(np.unique(sexo_sub)).issubset({0,1}), "Sexo debe ser 0/1."
n_muni_sub = df_sub["municipio_idx_sub"].nunique()
assert muni_sub.min() >= 0 and muni_sub.max() < n_muni_sub, "municipio_idx_sub fuera de rango."

print(f"n_obs={len(y_sub)} | n_muni_sub={n_muni_sub}")
print("Arrays listos: y_sub, edad_sub, sexo_sub, edad_c_sub, edad_cs_sub, muni_sub")


n_obs=6000 | n_muni_sub=177
Arrays listos: y_sub, edad_sub, sexo_sub, edad_c_sub, edad_cs_sub, muni_sub


In [27]:
# ====== B) B-splines cúbicos para edad ======

# Grados de libertad (ajústalo: 5 suele ir bien para modo rápido)
df_spline = 5

# Construcción de la matriz de bases spline sobre la edad OBSERVADA en la submuestra
X_spline_sub = dmatrix(
    f"bs(edad, df={df_spline}, degree=3, include_intercept=False) - 1",
    {"edad": edad_sub},
    return_type="dataframe"
)

# A NumPy (PyMC espera arrays)
X_s_sub = X_spline_sub.to_numpy()

# Tamaños para control
n_obs_sub, n_s_sub = X_s_sub.shape
print("X_s_sub shape =", X_s_sub.shape)    # (n_obs_sub, n_bases)

# Chequeo: debe haber una FILA por observación
assert n_obs_sub == len(y_sub), "X_s_sub debe tener n_obs_sub filas (una por observación)."


X_s_sub shape = (6000, 5)


---

## Sandbox

---